In [1]:
import os
from openai import OpenAI
import comet_llm
from IPython.display import display
import ipywidgets as widgets
import time
import dotenv 

In [2]:
# Input API keys

MY_OPENAI_KEY = os.getenv('MY_OPENAI_KEY')
MY_COMET_KEY = os.getenv('MY_COMET_KEY')


comet_llm.init(project="ecom_shopbot_openai",
               api_key=MY_COMET_KEY)

COMET INFO: Valid Comet API Key saved in /home/codespace/.comet.config (set COMET_CONFIG to change where it is saved).


In [3]:
# Customize your product list if necessary.
product_list = '''
# Fashion Shop Product List



- Perfumes  
  - Price: $20  
  - Available Sizes: Small, Medium, Large, XL  
  - Available Colors: Red, White, Black, Gray, Navy
  
- Mobile  
  - Price: $150  
  - Available Sizes: Small, Medium, Large, XL  
  - Available Colors: Black, Navy Blue, Burgundy

- Gifts  
  - Price: $45  
  - Available Sizes: Small, Medium, Large  
  - Available Colors: Floral Print, Blue, Pink

- Laptops  
  - Price: $80  
  - Available Sizes: Small, Medium, Large, XL  
  - Available Colors: Black, Gray, Navy

  - SkinCare 
  - Price: $80  
  - Available Sizes: Small, Medium, Large, XL  
  - Available Colors: Black, Gray, Navy



context = [{'role': 'system',
            'content': f"""
You are ShopBot, an AI assistant for my online fashion shop - Nhi Yen. 

Your role is to assist customers in browsing products, providing information, and guiding them through the checkout process. 

Be friendly and helpful in your interactions.

We offer a variety of products across categories such as Perfumes, Iphone, Skincsre, Laptops, mobile and key holders. 

Feel free to ask customers about their preferences, recommend products, and inform them about any ongoing promotions.

The Current Product List is limited as below:

```{product_list}```

Make the shopping experience enjoyable and encourage customers to reach out if they have any questions or need assistance.
"""}]



In [4]:
# Create a Chatbot
def get_completion_from_messages(messages, model="gpt-3.5-turbo"):
    client = OpenAI(
        api_key=MY_OPENAI_KEY,
    )

    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
    )
    return chat_completion.choices[0].message.content

In [5]:
# Interacting with Customers

def collect_messages(_):
    user_input = inp.value
    inp.value = ''
    context.append({'role':'user', 'content':f"{user_input}"})

    # Record the start time
    start_time = time.time()  

    response = get_completion_from_messages(context) 

    # Record the end time
    end_time = time.time()  

    # Calculate the duration
    duration = end_time - start_time  

    # Log to comet_llm
    comet_llm.log_prompt(
        prompt=user_input,
        output=response,
        duration=duration,
        metadata={
        "role": context[-1]['role'],
        "content": context[-1]['content'],
        "context": context,
        "product_list": product_list
        },
    )

    context.append({'role': 'assistant', 'content': f"{response}"})

    user_pane = widgets.Output()
    with user_pane:
        display(widgets.HTML(f"<b>User:</b> {user_input}"))

    assistant_pane = widgets.Output()
    with assistant_pane:
        display(widgets.HTML(f"<b>Assistant:</b> {response}"))

    display(widgets.VBox([user_pane, assistant_pane]))

inp = widgets.Text(value="Hi", placeholder='Enter text here…')
button_conversation = widgets.Button(description="Chat!")
button_conversation.on_click(collect_messages)

dashboard = widgets.VBox([inp, button_conversation])

display(dashboard)


Prompt logged to https://www.comet.com/yennhi95zz/ecom-shopbot-openai


In [6]:
messages =  context.copy()
messages.append(
{'role':'system'
 , 'content':'create a table summary of the previous order.'}   
)

response = get_completion_from_messages(messages)
print(response)


Certainly! Here's a summary of your order:

| Product             | Color | Size | Price |
|---------------------|-------|------|-------|
| Red Women's T-shirt | Red   | Small| $20   |
| Black Running Shoes | Black | 10   | $40   |

If everything looks good, you can proceed to checkout. Let me know if you need any further assistance!
